In [2]:
import numpy as np
import itertools as itr

import os as os
import sys as sys 
import pandas as pd
import warnings
sys.path.append(os.path.join("..","Libraries","QML_lib"))
import Evo as evo

global paulis_list
paulis_list = {'i' : np.eye(2), 'x' : evo.sigmax(), 'y' : evo.sigmay(), 'z' : evo.sigmaz()}

import hashlib

# Naming Convention
### Definitions

In [5]:
class operator():
    # Better to have this info in a DB than recalculating when called?
    def __init__(self, name): 
        self.name = name
    
    @property
    def constituents_names(self):
        t_str, p_str, max_t, max_p = get_t_p_strings(self.name)
        paulis_list = {'i' : np.eye(2), 'x' : evo.sigmax(), 'y' : evo.sigmay(), 'z' : evo.sigmaz()}
        if(max_t >= max_p):
            #constituent_names.append(self.name)
            return [self.name]
        else: 
            return self.name.split(p_str)
            #constituent_names = self.name.split(p_str)
        #return constituent_names        

    @property
    def num_qubits(self):
        return get_num_qubits(self.name)
        
    @property
    def constituents_operators(self):
        ops = []
        for i in self.constituents_names:
            ops.append(compute(i))
        return ops

    @property
    def num_constituents(self):
        # 1 param per constituent?
        return len(self.constituents_names)
    
    @property 
    def matrix(self):
        mtx = empty_array_of_same_dim(self.name)
        #dim = 2**self.num_qubits
        #print("Constructing matrix of dim: ", dim)
        #mtx = np.zeros([dim, dim], np.complex128)
        for i in self.constituents_operators:
            mtx += i
        return mtx
    @property
    def qubits_acted_on(self):
        return list_used_qubits(self.name)
   
    @property 
    def two_to_power_used_qubits_sum(self):
        running_sum = 0
        for x in list_used_qubits(self.name):
            running_sum += 2**x
        return running_sum

    @property
    def alph_name(self):
        return alph(self.name)
    
    
    def test_const(self):
        for i in self.constituents_operators:
            print(i)
            
    
    def interactions_on(self):
        # work out which qubits have actions; list ?
        return 0

def get_num_qubits(name):
    max_t_found = 0 
    t_str=''
    while name.count(t_str+'T')>0:
        t_str=t_str+'T'

    num_qubits = len(t_str) + 1
    return num_qubits
    

def list_used_qubits(name):
    max_t, t_str = find_max_letter(name, "T")
    max_p, p_str = find_max_letter(name, "P")
    running_list = []

    if max_p >= max_t:
        list_by_p_sep = []
        sep_by_p = name.split(p_str)
        for x in sep_by_p:
            list_by_p_sep.append(get_acted_on_qubits(x))

        for i in range(len(list_by_p_sep)):
            to_add= list(set(list_by_p_sep[i]) - set(running_list))
            running_list = running_list + to_add

    else:
        running_list = get_acted_on_qubits(name)
    return running_list


def get_acted_on_qubits(name):
    max_t, t_str = find_max_letter(name, "T")
    max_p, p_str = find_max_letter(name, "P")
    if max_p > max_t:
        list_by_p_sep = []
        sep_by_p = name.split(p_str)
        for x in sep_by_p:
            list_by_sep.append(fill_qubits_acted_on_list, x)
    
    
    qubits_acted_on = []
    fill_qubits_acted_on_list(qubits_acted_on,name)
    return sorted(qubits_acted_on)
    
def fill_qubits_acted_on_list(qubits_acted_on, name):
    max_t, t_str = find_max_letter(name, "T")
    max_p, p_str = find_max_letter(name, "P")
    if(max_p > max_t):
        string_to_analyse = name.split(p_str)[0]
    else:
        string_to_analyse = name

    if max_t == 0:
        if string_to_analyse != 'i':
            qubits_acted_on.append(1)


    else:
        i=max_t
        this_t_str = t_str
        broken_down = string_to_analyse.split(this_t_str)
        lhs = broken_down[0]
        rhs = broken_down[1]
        if rhs !='i':
            qubits_acted_on.append(i+1)

        if max_t == 1:
            if lhs!='i':
                qubits_acted_on.append(1)
        else: 
            fill_qubits_acted_on_list(qubits_acted_on, lhs)                
    
def get_t_p_strings(name):
    t_str = ''
    p_str = ''
    while name.count(t_str+'T')>0:
        t_str=t_str+'T'

    while name.count(p_str+'P')>0:
        p_str=p_str+'P'

    max_t = len(t_str)
    max_p = len(p_str)

    return t_str, p_str, max_t, max_p        
    
def find_max_letter(string, letter):
    letter_str=''
    while string.count(letter_str+letter)>0:
        letter_str=letter_str+letter

    return len(letter_str), letter_str


def empty_array_of_same_dim(name):
    t_str=''
    while name.count(t_str+'T')>0:
        t_str=t_str+'T'

    num_qubits = len(t_str) +1
    dim = 2**num_qubits
    #print("String: ", name, " has NQubits: ", num_qubits)
    empty_mtx = np.zeros([dim, dim], dtype=np.complex128)
    return empty_mtx



def alph(name):
    t_max, t_str = find_max_letter(name, "T")
    p_max, p_str = find_max_letter(name, "P")
    m_max, m_str = find_max_letter(name, "M")
    
    if p_max == 0 and t_max ==0 and p_max ==0 :
        return name
    
    if p_max > t_max and p_max > m_max: 
        ltr = 'P'
        string = p_str
    elif t_max >= p_max:
        string = t_str
        ltr = 'T'
    elif m_max >= p_max: 
        string = m_str
        ltr = 'M'
    elif t_max > m_max: 
        string = t_str
        ltr = 'T'
    else:
        ltr = 'M'
        string = m_str

    spread = name.split(string)
    if  p_max==m_max and p_max > t_max:
        string = p_str
        list_elements = name.split(p_str)
        
        for i in range(len(list_elements)):
            list_elements[i] = alph(list_elements[i])
        sorted_list = sorted(list_elements)
        linked_sorted_list = p_str.join(sorted_list)
        return linked_sorted_list
        
    if ltr=='P' and p_max==1:
        sorted_spread = sorted(spread)
        out = string.join(sorted_spread)
        return out
    elif ltr=='P' and p_max>1:
        list_elements = name.split(string)
        sorted_list = sorted(list_elements)
        for i in range(len(sorted_list)):
            sorted_list[i] = alph(sorted_list[i])
        linked_sorted_list = string.join(sorted_list)
        return linked_sorted_list
    else: 
        for i in range(len(spread)):
            spread[i] = alph(spread[i])
        out = string.join(spread)
        return out


def compute_t(inp):
   # print("Compute t : ", inp)
    max_t, t_str = find_max_letter(inp, "T")
    max_p, p_str = find_max_letter(inp, "P")

    if(max_p == 0 and max_t==0):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 

    elif(max_t==0):
        return compute(inp)
    else:
        to_tens = inp.split(t_str)
        #print("To tens: ", to_tens)
        running_tens_prod=compute(to_tens[0])
        #print("Split by ", t_str, " : \n", to_tens)
        for i in range(1,len(to_tens)):
            max_p, p_str = find_max_letter(to_tens[i], "P")
            max_t, t_str = find_max_letter(to_tens[i], "T")
            #print("To tens [i=", i, "]:\n", to_tens[i] )
            rhs = compute(to_tens[i])
            running_tens_prod = np.kron(running_tens_prod, rhs)
        #print("RESULT ", t_str, " : ", inp, ": \n", running_tens_prod)
        return running_tens_prod

def compute_p(inp):
    #print("Compute p : ", inp)
    max_p, p_str = find_max_letter(inp, "P")
    max_t, t_str = find_max_letter(inp, "T")

    if(max_p == 0 and max_t==0):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 

    elif max_p==0:
        return compute(inp)
    else: 
        to_add = inp.split(p_str)
        #print("To add : ", to_add)
        running_sum = empty_array_of_same_dim(to_add[0])
        for i in range(len(to_add)):
            max_p, p_str = find_max_letter(to_add[i], "P")
            max_t, t_str = find_max_letter(to_add[i], "T")

           # print("To add [i=", i, "]:", to_add[i] )
            rhs = compute(to_add[i])
            #print("SUM shape:", np.shape(running_sum))
            #print("RHS shape:", np.shape(rhs))
            running_sum += rhs

        #print("RESULT ", p_str, " : ", inp, ": \n", running_sum)
        return running_sum


def compute_m(inp):
    max_m, m_str = find_max_letter(inp, "M")
    max_p, p_str = find_max_letter(inp, "P")
    max_t, t_str = find_max_letter(inp, "T")

    if(max_m == 0 and max_t==0 and max_p == 0 ):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 

    elif max_m ==0:
        return compute(inp)
    
    else:   
        to_mult = inp.split(m_str)
        #print("To mult : ", to_mult)
        t_str=''
        while inp.count(t_str+'T')>0:
            t_str=t_str+'T'

        num_qubits = len(t_str) +1
        dim = 2**num_qubits

        running_product = np.eye(dim)

        for i in range(len(to_mult)):
            #print("Running product : \n", running_product, "\n times \n", compute(to_mult[i]))
            running_product = np.dot(running_product, compute(to_mult[i]))

        return running_product    
    
def compute(inp):
    #print("Computing ", inp)
    max_p, p_str = find_max_letter(inp, "P")
    max_t, t_str = find_max_letter(inp, "T")
    max_m, m_str = find_max_letter(inp, "M")

    if(max_m == 0 and max_t==0 and max_p == 0):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 
    elif max_m > max_t:
        return compute_m(inp)
    elif max_t >= max_p:
       # print("Max t=", max_t, ">= max p=", max_p)
        return compute_t(inp)
    else:
        return compute_p(inp)    


# Database

In [30]:
def launch_db(RootN_Qbit=[0], N_Qubits=1, gen_list=[]):
    
    # Create DB using strings of names
    generators = []
    total_model_list = []
    Max_N_Qubits = 11
    model_lists = {}
    for j in range(1, Max_N_Qubits):
        model_lists[j] = []
    
    for i in gen_list:
        generators.append(operator(i))
        alph_model_name = alph(i)
        num_qubits = get_num_qubits(i)
        model_lists[num_qubits].append(alph_model_name)

    model_db = pd.DataFrame({
        '<Name>' : [ gen.name for gen in generators], 
        'Alph_Name' :[ gen.alph_name for gen in generators],
        'All_Operators_Names': [gen.constituents_names for gen in generators],
        'All_Operators_Matrices': [gen.constituents_operators for gen in generators],
        'N_params' : [ gen.num_constituents for gen in generators ],
        'Num_Qubits' : [ gen.num_qubits for gen in generators ],
        'Binary_Sum_Used_Qubits' : [ gen.two_to_power_used_qubits_sum for gen in generators ],
        'Matrix' : [gen.matrix for gen in generators]
    })
        
    # if N_qubits defined: work out generator list.
    # Or should number qubits be implied by gen list?
    db = pd.DataFrame({
        '<Name>' : [ gen.name for gen in generators], 
        'Alph_Name' :[ gen.alph_name for gen in generators],
        'Qubits_Acted_On' : [ gen.qubits_acted_on for gen in generators ],
        'DB_location' : [ get_location(model_db, gen.name) for gen in generators],
        'Status' : 'Ready', 
        'Selected' : False, 
        'LogL_Ext' : None, 
        'QML_Class' : None, 
        'Origin_epoch' : 0, 
        'RootNode' : 'NaN',
        })  
        
    return db, model_db, model_lists

def get_location(db, name):
    for i in range(len(db['<Name>'])):
        if db['<Name>'][i] == name:
            return i

def get_location_by_alph_name(db, name):
    for i in range(len(db['Alph_Name'])):
        if db['Alph_Name'][i] == name:
            return i

        
def consider_new_model(model_lists, name, db):
    # Return true indicates it has not been considered and so can be added
    al_name = alph(name)
    n_qub = get_num_qubits(name)
    if al_name not in model_lists[n_qub]:
        return 'New'
    else: 
        location = get_location_by_alph_name(db, al_name)
        return location
        
def add_model(model_name, running_database , model_db = model_db, model_lists = model_lists):
    ## Alternatively, can use existing model_db and total_model_list
    ## What is preferred?
    alph_model_name = alph(model_name)
    model_num_qubits = get_num_qubits(model_name)
    
    if consider_new_model(model_lists, model_name, running_database)== 'New':
        print("Model Not previously considered -- adding")
        op = operator(model_name)
        num_rows = len(running_database)

        # add model_db_new_row to model_db and running_database
        # Note: do NOT use pd.df.append() as this copies total DB,
        # appends and returns copy.
        
        model_db_new_row = pd.Series({
            '<Name>': op.name, 
            'Alph_Name': op.alph_name,
            'All_Operators_Names' : op.constituents_names,
            'All_Operators_Matrices': op.constituents_operators,
            'N_params' : op.num_constituents,
            'Num_Qubits' : op.num_qubits,
            'Binary_Sum_Used_Qubits' : op.two_to_power_used_qubits_sum,
            'Matrix' : op.matrix
            })

        model_db.loc[num_rows] = model_db_new_row
       # total_model_list.append(alph_model_name)
        model_lists[model_num_qubits].append(alph_model_name)
        
        # Add to running_database too, after adding to model_db
        
        running_db_new_row = pd.Series({
            '<Name>': op.name,
            'Alph_Name' : op.alph_name,
            'Qubits_Acted_On' : op.qubits_acted_on,
            'DB_location' : get_location(model_db, model_name),
            'Status' : 'Ready', 
            'Selected' : False, 
            'LogL_Ext' : None, 
            'QML_Class' : None, 
            'Origin_epoch' : 0, 
            'RootNode' : 'NaN',
        })

        running_database.loc[num_rows] = running_db_new_row      
        
    else:
        location = consider_new_model(model_lists, model_name, running_database)
        #db_loc = get_location_by_alph_name(model_db, model_name)
        print("Model", alph_model_name, " previously considered at location", location)  

In [31]:
model_lists

{1: ['xPz'],
 2: ['yTxPPzTi', 'xTy'],
 3: [],
 4: ['xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi'],
 5: [],
 6: [],
 7: [],
 8: [],
 9: [],
 10: []}

## Example usage of database

In [32]:
name1 = 'yTxPPzTi'
name2 = 'xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi'
name3 = 'xTy'

gen_list=[name1, name2, name3]
db, model_db, model_lists = launch_db(gen_list=gen_list)


In [33]:
db

,<Name>,Alph_Name,DB_location,LogL_Ext,Origin_epoch,QML_Class,Qubits_Acted_On,RootNode,Selected,Status
0,yTxPPzTi,yTxPPzTi,0,None,0,None,"[1, 2]",NaN,False,Ready
1,xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi,xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi,1,None,0,None,"[1, 2, 4]",NaN,False,Ready
2,xTy,xTy,2,None,0,None,"[1, 2]",NaN,False,Ready


In [25]:
model_db

,<Name>,All_Operators_Matrices,All_Operators_Names,Alph_Name,Binary_Sum_Used_Qubits,Matrix,N_params,Num_Qubits,Qubits_Acted_On
0,yTxPPzTi,"[[[0j, 0j, 0j, -1j], [0j, 0j, -1j, 0j], [0j, 1...","[yTx, zTi]",yTxPPzTi,6,"[[(1+0j), 0j, 0j, -1j], [0j, (1+0j), -1j, 0j],...",2,2,"[1, 2]"
1,xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi,"[[[0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-1-0j)...","[xTyPPyTiTTTiTyPPxTi, xTyPPyTiTTTiTyPPxTi]",xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi,22,"[[0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-2+0j),...",2,4,"[1, 2, 4]"
2,xTy,"[[[0j, 0j, 0j, -1j], [0j, 0j, 1j, 0j], [0j, -1...",[xTy],xTy,6,"[[0j, 0j, 0j, -1j], [0j, 0j, 1j, 0j], [0j, -1j...",1,2,"[1, 2]"


Example: Try to add 

yTiPPxTyTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi

which is equivalent to:

xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi

In [26]:
add_model('yTiPPxTyTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi', db, model_db, model_lists)

Model xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi  previously considered at location 1


Example: add xPz

In [27]:
add_model('xPz', db, model_db, model_lists)

Model Not previously considered -- adding


In [28]:
# After: 
db

,<Name>,Alph_Name,DB_location,LogL_Ext,N_Qbit,Origin_epoch,QML_Class,RootNode,Selected,Status
0,yTxPPzTi,yTxPPzTi,0,None,[0],0,None,NaN,False,Ready
1,xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi,xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi,1,None,[0],0,None,NaN,False,Ready
2,xTy,xTy,2,None,[0],0,None,NaN,False,Ready
3,xPz,xPz,3,None,None,0,None,NaN,False,Ready


In [29]:
# After: 
model_db

,<Name>,All_Operators_Matrices,All_Operators_Names,Alph_Name,Binary_Sum_Used_Qubits,Matrix,N_params,Num_Qubits,Qubits_Acted_On
0,yTxPPzTi,"[[[0j, 0j, 0j, -1j], [0j, 0j, -1j, 0j], [0j, 1...","[yTx, zTi]",yTxPPzTi,6,"[[(1+0j), 0j, 0j, -1j], [0j, (1+0j), -1j, 0j],...",2,2,"[1, 2]"
1,xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi,"[[[0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-1-0j)...","[xTyPPyTiTTTiTyPPxTi, xTyPPyTiTTTiTyPPxTi]",xTyPPyTiTTTiTyPPxTiPPPPxTyPPyTiTTTiTyPPxTi,22,"[[0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-2+0j),...",2,4,"[1, 2, 4]"
2,xTy,"[[[0j, 0j, 0j, -1j], [0j, 0j, 1j, 0j], [0j, -1...",[xTy],xTy,6,"[[0j, 0j, 0j, -1j], [0j, 0j, 1j, 0j], [0j, -1j...",1,2,"[1, 2]"
3,xPz,"[[[0j, (1+0j)], [(1+0j), 0j]], [[(1+0j), 0j], ...","[x, z]",xPz,2,"[[(1+0j), (1+0j)], [(1+0j), (-1+0j)]]",2,1,[1]


In [17]:
# Try adding zPx 
add_model('zPx', db, model_db, model_lists)

Model xPz  previously considered at location 3
